In [1]:
import pandas as pd
import numpy as np
import sqlite3 as mysql
import time
import re
from datetime import datetime

In [2]:
conn = mysql.connect(r"D:\SQLBASE\weather.db")

In [3]:
cur = conn.cursor()

In [4]:
cur.execute('select count(ymd) from history_tem_right_20110101_20181225')
total_rows = cur.fetchone()[0]

In [2]:
conn2 = mysql.connect(r"D:\SQLBASE\WeatherBefore.db")
curs = conn2.cursor()

In [107]:
cur.execute('select * from history_tem_right_20110101_20181225')
haved_df = pd.read_sql('select city_ymd from WeatherAll', conn2)
haved_df.drop_duplicates(inplace=True)
col_name_list = [tuple[0] for tuple in cur.description]
for index in range(1, total_rows//100000+2):
    print('第{}个10万数据正在访问'.format(index))
    million_piece = cur.fetchmany(100000)
    million_df = pd.DataFrame(million_piece, columns=col_name_list)
    
    million_df = million_df.loc[million_df['city'] != None].copy()

    million_df['city_ymd'] = million_df['city'] + '-' + million_df['ymd'].apply(lambda x: str(x))
    million_df = million_df[~million_df['city_ymd'].isin(haved_df['city_ymd'].tolist())]
    if not million_df.empty:
        million_df['ymd_time'] = million_df['ymd'].apply(lambda x: datetime.strptime(str(x), '%Y%m%d'))
        million_df['year'] = million_df['year'].apply(lambda x: str(x))
        million_df['month'] = million_df['ymd'].apply(lambda x: str(x)[4:-2])
        million_df['day'] = million_df['ymd'].apply(lambda x: str(x)[-2:])

        million_df['max_tm'] = million_df['max_tm'].apply(lambda x: int(x) if x is not None and x.isdigit() else np.NaN)
        million_df['min_tm'] = million_df['min_tm'].apply(lambda x: int(x) if x is not None and x.isdigit() else np.NaN)
        million_df['weekday'] = million_df['ymd_time'].dt.weekday + 1

        million_df.drop(columns=['ymd_time'], inplace=True)

        million_df['ave_tm'] = (million_df['max_tm'] + million_df['min_tm']) * 0.5
        million_df.drop_duplicates(inplace=True)
        try:
            million_df.to_sql(
                name='WeatherAll',
                con=conn2,
                chunksize=100000,
                if_exists = 'append',
                index=False,
                )
            conn2.commit()
        except mysql.IntegrityError as e:
            print('数据已经写过，跳过\n', e)

第1个10万数据正在访问
第2个10万数据正在访问
第3个10万数据正在访问
第4个10万数据正在访问
第5个10万数据正在访问
第6个10万数据正在访问
第7个10万数据正在访问
第8个10万数据正在访问
第9个10万数据正在访问
第10个10万数据正在访问
第11个10万数据正在访问
第12个10万数据正在访问
第13个10万数据正在访问
第14个10万数据正在访问
第15个10万数据正在访问
第16个10万数据正在访问
第17个10万数据正在访问
第18个10万数据正在访问
第19个10万数据正在访问
第20个10万数据正在访问
第21个10万数据正在访问
第22个10万数据正在访问
第23个10万数据正在访问
第24个10万数据正在访问
第25个10万数据正在访问
第26个10万数据正在访问
第27个10万数据正在访问
第28个10万数据正在访问
第29个10万数据正在访问
第30个10万数据正在访问
第31个10万数据正在访问
第32个10万数据正在访问
第33个10万数据正在访问
第34个10万数据正在访问
第35个10万数据正在访问
第36个10万数据正在访问
第37个10万数据正在访问
第38个10万数据正在访问
第39个10万数据正在访问
第40个10万数据正在访问
第41个10万数据正在访问
第42个10万数据正在访问
第43个10万数据正在访问
第44个10万数据正在访问
第45个10万数据正在访问
第46个10万数据正在访问
数据已经写过，跳过
 UNIQUE constraint failed: WeatherAll.city_ymd
第47个10万数据正在访问
第48个10万数据正在访问
第49个10万数据正在访问
第50个10万数据正在访问
第51个10万数据正在访问
第52个10万数据正在访问
第53个10万数据正在访问
第54个10万数据正在访问
第55个10万数据正在访问
第56个10万数据正在访问
第57个10万数据正在访问
第58个10万数据正在访问
第59个10万数据正在访问
第60个10万数据正在访问
数据已经写过，跳过
 UNIQUE constraint failed: WeatherAll.city_ymd
第61个10万数据正在访问
数据已经写过，跳过
 UNIQUE constraint failed: Weat

In [18]:
conn2.commit()

In [19]:
conn2.close()

In [20]:
a = '2'
a.isdigit()

True

In [115]:
curs.execute('select * from WeatherAll where city_ymd ISNULL')

In [13]:
# curs.fetchall()

In [3]:
config_df = pd.read_csv('select_month.csv')

In [4]:
config_df['time'] = config_df.apply(lambda x: pd.Timestamp(int(x['year']), int(x['month']), 1), axis=1)

In [6]:
time_df = pd.DataFrame(pd.date_range(start='20110101', end='20301231', freq='M'))

In [7]:
time_df['time'] = time_df[0].apply(lambda x: datetime.strftime(x, '%Y%m'))

In [8]:
config_df['time'] = config_df['time'].apply(lambda x: datetime.strftime(x, '%Y%m'))

In [9]:
total_cities = []
for one_city in config_df['city'].unique():
    time_df2 = time_df.copy()
    time_df2['city'] = one_city
    total_cities.append(time_df2)

In [10]:
total_city_df = pd.concat(total_cities, sort=False)

In [12]:
total_city_df.columns = ['date', 'year_month', 'city']

In [16]:
total_city_df['city_year_month'] = total_city_df['city'] + '_' + total_city_df['year_month'].map(str)

In [17]:
total_city_df.to_sql(name='config_date_city',
                con=conn2,
                chunksize=100000,
                if_exists='append',
                index=False)

In [170]:
result_df = config_df.merge(total_city_df, on=['city', 'time'], how='outer')

In [176]:
need_more_df = result_df.loc[(result_df[0] < pd.Timestamp(2019,11, 11)) & (result_df['count'].isna())].copy()

In [178]:
need_more_df[['city', 'time']].to_clipboard()

In [180]:
need_more_df['weather_url'] = need_more_df.apply(lambda x: "http://www.tianqihoubao.com/lishi/%s/month/%s.html" % (x['city'], x['time']),axis=1)

In [190]:
aa = need_more_df[['city', 'time', 'weather_url']]

In [202]:
aa.set_index(['city', 'time']).to_dict()['weather_url']

'http://www.tianqihoubao.com/lishi/aba/month/201908.html'

In [3]:
df = pd.read_excel(r"E:\文档\Python\VisualCode\工作交接\天气爬虫-d\后报\数据爬取\city_config.xls")

In [4]:
df.head()

,city,area,s_area,area_name,provi
0,aba,南大区,西南,阿坝,四川
1,abagaqi,华北大区,北一,阿巴嘎旗,内蒙古
2,acheng,北大区,东北,阿城,黑龙江
3,aershan,华北大区,北一,阿尔山,内蒙古
4,aheqi,北大区,西北,阿合奇,新疆


In [5]:
df.drop_duplicates(inplace=True)

In [7]:
df['city'].nunique()

2388

In [8]:
df.shape

(2388, 5)

In [14]:
df.to_sql(name='config_city_area',
          con=conn2,
          if_exists='append',
          index=False,
          chunksize=100000,
#           method='multi'
         )